# Deployment of a trained model that had been stored in IBM Watson Machine Learning Service

In [7]:
from ibm_watson_machine_learning import APIClient


credentials = {
    "apikey":"",
    # the WML was created in london region #
    "url":"https://eu-gb.ml.cloud.ibm.com"
}

wml_client = APIClient(credentials)
list = wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------  ------------------------
ID                                    NAME          CREATED
aae29536-2d1b-46d5-8c3b-ba01545330a1  Model_deploy  2023-11-12T18:17:47.028Z
------------------------------------  ------------  ------------------------


# Getting creds: 
Please refer to this Github file [`Store_model.ipynb`](https://github.com/yaghmo/store_deploy_CNN_model_IBM_cloud/blob/main/Store_model.ipynb)

In [122]:
space_name = 'Model_deploy'
space_guid = list[list['NAME']==space_name]['ID'][0]
wml_client.set.default_space(space_guid)
models_details = wml_client._models.list()

------------------------------------  --------------------  ------------------------  ---------------  ----------  ----------------
ID                                    NAME                  CREATED                   TYPE             SPEC_STATE  SPEC_REPLACEMENT
39df5cf2-3706-457c-8ab2-abf538fd807e  jellyfish_classifier  2023-11-13T14:00:40.002Z  tensorflow_2.12  supported
------------------------------------  --------------------  ------------------------  ---------------  ----------  ----------------


In [124]:
# name of your model #
model_name = 'jellyfish_classifier'
model_uid = models_details[models_details['NAME']==model_name]['ID'][0]

In [125]:
# Set meta #
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:space_name, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# Deploy #
deployment = wml_client.deployments.create(
    artifact_uid=model_uid, 
    meta_props=deployment_props 
)

'''NOTE: you can print the deployment variable to see its details but if no error is shows it should be working'''



#######################################################################################

Synchronous deployment creation for uid: '39df5cf2-3706-457c-8ab2-abf538fd807e' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='820ffddc-d1ed-4cb4-801c-12f664ecc153'
------------------------------------------------------------------------------------------------




'NOTE: you can print the deployment variable to see its details but if no error is shows it should be working'

In [131]:
# getting 1 sample input #
from PIL import Image
import tensorflow as tf
import numpy as np
im_height, im_width = 256, 256

# same as used for training the model #
def preprocess(image):
    image = tf.image.resize(image, (im_height, im_width))
    image = image / 255.0  
    image =  np.array(image)
    return image

# list of your images, in this case randomly from internet #
images = ['./moon.jpg','./barrel.jpg','./compass.jpg','./lion.jpg','./mauve.jpg','./blue.jpg']

input = []
for image in images:
    input.append(preprocess(Image.open(image)))
# NOTE : the input has to be a list of numpy : even if you used a Dataframe just do pd.to_numpy().to_list() to your input df #
input = np.array(input).tolist()

In [128]:
deployment_uid = wml_client.deployments.get_uid(deployment)

# preparing the inputs
payload = {"input_data":
           [    
               {"values":input}
           ]
          }
results = wml_client.deployments.score(deployment_uid, payload)['predictions'][0]

#### The model only recognize one of these 6 jellyfish types:
- Moon jellyfish
- barrel jellyfish
- blue jellyfish
- compass jellyfish
- lions mane jellyfish
- mauve stinger jellyfish


In [130]:
# the types we have they need to be sorted as the same way they been sorted in species variable during the cnn model training #
types = ['Moon jellyfish', 'barrel jellyfish', 'blue jellyfish', 'compass jellyfish', 'lions mane jellyfish', 'mauve stinger jellyfish']
# types = ['Moon_jellyfish', 'barrel_jellyfish', 'blue_jellyfish', 'compass_jellyfish', 'lions_mane_jellyfish', 'mauve_stinger_jellyfish']

for result,image in zip(results['values'],images):
    print('The image {} is a {} with the score of {:.2f}%'.format(image,types[np.argmax(result)],np.max(result)))

The image ./moon.jpg is Moon_jellyfish with the score of 1.00%
The image ./barrel.jpg is barrel_jellyfish with the score of 0.96%
The image ./compass.jpg is compass_jellyfish with the score of 0.94%
The image ./lion.jpg is compass_jellyfish with the score of 0.63%
The image ./mauve.jpg is compass_jellyfish with the score of 0.64%
The image ./blue.jpg is Moon_jellyfish with the score of 0.98%
